# Imports

In [2]:
from utils.setseed import set_seed
set_seed(1234)

import os

import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from torch import optim, nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from tqdm import tqdm

from data.dataset import MyDataset
from model.STTN import STTN
from model.STTN_feature import STTN_feature
from utils.earlystoping import EarlyStopping
from utils.getdata import get_data
from helper import import_data_new, get_data_train_no_X, get_adj


# Preparing Data

In [9]:
config = {
    'his_len':3,
    'pre_len':1,
    'batch_size':3,
    'lr':1e-3,

}

In [3]:
A = get_adj('adjacency_matrixdjace.npy')


In [5]:
file_path = 'school_weekly_weather_traffic2(1).csv'
df = import_data_new(file_path, train_test='train', weather='Yes',traffic='Yes',season_dummy='Yes')
train = get_data_train_no_X(df, target_name='Avg_Delay_Time')
mean = torch.mean(train).item()
std = torch.std(train).item()
train = (train - mean) / std

df_test = import_data_new(file_path, train_test='test', weather='Yes',traffic='Yes',season_dummy='Yes')
test = get_data_train_no_X(df_test, target_name='Avg_Delay_Time')
test = (test - mean) / std

/Users/sundemiao/Library/CloudStorage/GoogleDrive-cleons@andrew.cmu.edu/My Drive/STTN2/helper.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/Users/sundemiao/Library/CloudStorage/GoogleDrive-cleons@andrew.cmu.edu/My Drive/STTN2/helper.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [8]:
trainset = MyDataset(data=train, his_len=config['his_len'], pre_len=config['pre_len'])
validset = MyDataset(data=test, his_len=config['his_len'], pre_len=config['pre_len'])
testset = MyDataset(data=test, his_len=config['his_len'], pre_len=config['pre_len'])

trainloader = DataLoader(trainset, batch_size=config['batch_size'], shuffle=True)
validloader = DataLoader(validset, batch_size=config['batch_size'], shuffle=True)
testloader = DataLoader(testset, batch_size=config['batch_size'], shuffle=False)

# Train Def and Config

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
model = STTN(adj=A, len_his=config['his_len'], len_pred=config['pre_len']).to(device)
optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=1e-4)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95 ** ((epoch - 1) // 2))
criterion = nn.L1Loss()

In [14]:
def one_batch(model, criterion, batch_x,batch_y):
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    output = model(batch_x)
    loss = criterion(output, batch_y)
    return loss, output

def save_model(model, optimizer,scheduler,epoch, loss, best_val_loss, path):
    torch.save({'model_state_dict':model.state_dict(),
                'optimizer_state_dict':optimizer.state_dict(),
                'scheduler_state_dict':scheduler.state_dict(),
                'epoch':epoch,
                'loss':loss})
    if loss<best_val_loss:
        torch.save({'model_state_dict':model.state_dict(),
                'optimizer_state_dict':optimizer.state_dict(),
                'scheduler_state_dict':scheduler.state_dict(),
                'epoch':epoch,
                'best_val_loss':best_val_loss})
        
def tqdm_bar(batch_bar,total_loss,optimizer):
    batch_bar.set_postfix(
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
    batch_bar.update()


In [ ]:
def train(model,scheduler,optimizer,)